In [85]:
# !conda install -c conda-forge pandas-profiling -y

In [86]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.utils import resample
import pandas_profiling as pp


    Apply the Random Forest algorithm to predict the TARGET_B. Please note that this column suffers from class imbalance. Fix the class imbalance using upsampling.
    Discuss the model predictions and it's impact in the bussiness scenario. Is the cost of a false positive equals to the cost of the false negative? How much the money the company will not earn because of missclassifications made by the model?
    Sklearn classification models are trained to maximize the accuracy. However, another error metric will be more relevant here. Which one? Please checkout make_scorer alongside with GridSearchCV in order to train the model to maximize the error metric of interest in this case.


In [87]:
categorical_df = pd.read_csv("Categorical.csv")
numerical_df = pd.read_csv("Numerical.csv")
target_df = pd.read_csv("Target.csv")

In [88]:
merged_df = pd.concat([categorical_df, numerical_df, target_df], axis=1)

In [89]:
# # Create a profile report >> not working, issues withmy conda version
# profile = pp.ProfileReport(merged_df)
# profile.to_notebook_iframe()

In [90]:
#extract X and create target B

X = merged_df.drop(columns=["TARGET_B"])
y = target_df["TARGET_B"]

In [91]:
# One-hot encode the categorical features
categorical_cols = [col for col in X.columns if X[col].dtype == "object"]
X = pd.get_dummies(X, columns=categorical_cols, drop_first=True)

In [92]:
# train test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [93]:
#fix class impbalance with upsampling :

#split majority and majority classes
majority_class = X_train[y_train == 0]
minority_class = X_train[y_train == 1]

# Upsample the minority 
minority_upsampled = resample(minority_class, replace=True, n_samples=len(majority_class), random_state=42)

# recombine the majority class with upsampled minority 
upsampled_X_train = pd.concat([majority_class, minority_upsampled])
upsampled_y_train = pd.Series([0] * len(majority_class) + [1] * len(minority_upsampled))

In [94]:
#initiate random_forest
rf_classifier = RandomForestClassifier(random_state=42)
rf_classifier.fit(upsampled_X_train, upsampled_y_train)

RandomForestClassifier(random_state=42)

In [95]:
#predicting target B
y_pred = rf_classifier.predict(X_test)

In [96]:
# Evaluate random forst's performance
confusion = confusion_matrix(y_test, y_pred)
report = classification_report(y_test, y_pred)

print("Confusion Matrix:")
print(confusion)
print("Classification Report:")
print(report)

Confusion Matrix:
[[18105     0]
 [   61   917]]
Classification Report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     18105
           1       1.00      0.94      0.97       978

    accuracy                           1.00     19083
   macro avg       1.00      0.97      0.98     19083
weighted avg       1.00      1.00      1.00     19083



In [97]:
# here the model seems to be very efficient, 
# maybe too much, but in a real world scenario the cost of a false positive vs a
# false negative can vary, dependeing on the cost associated with either false
# positive/negative,or the business objective.

In [98]:
from sklearn.metrics import make_scorer, precision_score
from sklearn.model_selection import GridSearchCV

In [ ]:
from tqdm import tqdm  # Import tqdm
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier

# Instantiate the RandomForestClassifier with default hyperparameters
estimator = RandomForestClassifier()

# Define the parameter grid to search over
param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

# Create the GridSearchCV object with tqdm progress bar
grid_search = GridSearchCV(estimator, param_grid, scoring='accuracy', cv=5)

# Fit the grid search to your data with tqdm progress bar
for _ in tqdm(range(1)):  # Adjust the range according to your needs
    grid_search.fit(upsampled_X_train, upsampled_y_train)

# Get the best model and its parameters
best_model = grid_search.best_estimator_


  0%|          | 0/1 [00:00<?, ?it/s]